In [1]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from collections import deque

from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class memory():

    def __init__(self, size):
        self.mem = deque(maxlen=size)

    def append_sample(self, state, action, reward, next_state, done):
        self.mem.append((state, action, reward, next_state, done))

    def get_train_data(self,batch_size):
        return random.sample(self.mem, batch_size)

    def __len__(self):
        return len(self.mem)



In [2]:
class NN(nn.Module):

    def __init__(self, state_size, action_size):
        super(NN,self).__init__()
        self.fc1 = nn.Linear(state_size, 32)
        self.fc2 = nn.Linear(32,32)
        self.out = nn.Linear(32, action_size)
        nn.init.xavier_normal_(self.fc1.weight)
        nn.init.xavier_normal_(self.fc2.weight)
        nn.init.xavier_normal_(self.out.weight)



    def forward(self,x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.out(x)
        return x
        

In [3]:
action = 0
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.

In [4]:
class Agent():
    def __init__(self, mem_size, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        
        self.epsilon = 1
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9
        self.batch_size = 64
        self.GAMMA =0.9
        
        self.worker_NN = NN(state_size, action_size).to(device)
        self.target_NN = NN(state_size, action_size).to(device)
        self.target_NN.load_state_dict(self.worker_NN.state_dict())
        
        self.optimizer = optim.Adam(self.worker_NN.parameters(),lr=0.001)
        self.loss_func = nn.MSELoss()
        self.mem = memory(mem_size)
    
    def get_action(self,state):
        if np.random.rand() <= self.epsilon:
            self.epsilon = self.epsilon * self.epsilon_decay
            return env.action_space.sample()
        else:
            state = torch.Tensor(state).to(device)
            a=self.worker_NN(state)
            out = torch.argmax(a)
            return  out.cpu().numpy()
    
    def train_model(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        
        mini_batch = self.mem.get_train_data(self.batch_size)
        
        states = np.zeros((self.batch_size, self.state_size))
        next_states = np.zeros((self.batch_size, self.state_size))
        actions, rewards, dones = [],[],[]
        
        for i in range(self.batch_size):
            states[i] = mini_batch[i][0]
            actions.append(mini_batch[i][1])
            rewards.append(mini_batch[i][2])
            next_states[i] = mini_batch[i][3]
            dones.append(mini_batch[i][4])
        
        states = torch.Tensor(states).to(device)
        next_states = torch.Tensor(next_states).to(device)
        rewards = torch.Tensor(rewards).to(device)
        
        q_eval = self.worker_NN(states)
        #q_eval = q_eval.max(1)[0]
        
        q_val = torch.zeros(self.batch_size).to(device)
        q_target = torch.zeros(self.batch_size).to(device)
        q_next = self.target_NN(next_states)
        
        '''
        for w in range(self.batch_size):
            q_val[w] = q_eval[w][actions[w]]
            if dones[w]:
                q_target[w] = rewards[w]
            else:
                #print(q_next[w])
                #print(q_next[w].max())
                q_target[w] = rewards[w] + (self.GAMMA * q_next[w].max())
        '''  
        for w in range(self.batch_size):
            q_val[w] = q_eval[w][actions[w]]
            
        
        q_target = rewards + (self.GAMMA * q_next.max(1)[0])
        
        #print("q_eval_size : {}".format(q_eval.shape))
        #print("q_target_size : {}".format(q_target.shape))
        
        #loss = F.mse_loss(q_eval, q_target)
        loss = F.mse_loss(q_val, q_target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        
    def weight_copy(self):
        self.target_NN.load_state_dict(self.worker_NN.state_dict())
            
    

In [5]:
agent = Agent(mem_size=2000, state_size=state_size, action_size =action_size)

In [6]:
for i_episode in range(1500):
    state = env.reset()
    score = 0
    for t in range(600):
        env.render()
        #action = env.action_space.sample()
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])

        reward = reward if not done or score > 499 else -100 #안끝났거나 score가 499보다 큰경우가 아니고서야 reward는 -100

        agent.mem.append_sample(state, action, reward, next_state, done)
        state=next_state
        score +=reward
        if len(agent.mem) > 100:
            agent.train_model()

        if done:
            if t+1 > 10:
                print("{} : Episode finished after {} timesteps and ".format(i_episode, t+1))
            if (i_episode % 10) == 0:
                agent.weight_copy()
            break
env.close()


0 : Episode finished after 15 timesteps and 

1 : Episode finished after 19 timesteps and 

2 : Episode finished after 23 timesteps and 

3 : Episode finished after 29 timesteps and 

4 : Episode finished after 61 timesteps and 

...

912 : Episode finished after 360 timesteps and 

913 : Episode finished after 315 timesteps and 

914 : Episode finished after 332 timesteps and 

915 : Episode finished after 312 timesteps and 

KeyboardInterrupt: 